# ビジネス関連法案の RAG 検索 (e-GOV 法令検索)


In [1]:
import cohere
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from typing import List, Dict
import time
import re
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
co = cohere.ClientV2(os.environ['COHERE_APIKEY'])

EGOV_SEARCH_URL = "https://elaws.e-gov.go.jp/api/1/lawlists/1"

/Users/aki/tech/cohere-playground/env/lib/python3.14/site-packages/cohere/core/pydantic_utilities.py:13: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.datetime_parse import parse_date as parse_date


# 2. e-Gov から法令情報を取得

In [6]:

def fetch_egov_laws(keyword: str = "", limit: int = 50) -> List[Dict]:
    """
    e-Gov APIから法令情報を取得
    
    Args:
        keyword: 検索キーワード
        limit: 取得する法令の最大数
    
    Returns:
        法令情報のリスト
    """
    laws = []
    
    try:
        # e-Gov API にリクエスト
        response = requests.get(EGOV_SEARCH_URL, timeout=30)
        response.raise_for_status()
        
        # XMLパース
        soup = BeautifulSoup(response.content, 'xml')
        law_items = soup.find_all('LawNameListInfo')
        
        for item in law_items[:limit]:
            law_num = item.find('LawNo')
            law_name = item.find('LawName')
            
            if law_num and law_name:
                laws.append({
                    'law_number': law_num.text.strip(),
                    'law_name': law_name.text.strip(),
                    'text': f"{law_name.text.strip()} ({law_num.text.strip()})"
                })
        
        print(f"✓ {len(laws)}件の法令情報を取得しました")
        
    except Exception as e:
        print(f"⚠ エラー: {e}")
        # サンプルデータをフォールバック
        laws = get_sample_laws()
    
    return laws

import json
def get_sample_laws() -> List[Dict]:
    """サンプル法令データ（API接続失敗時のフォールバック）"""
    with open('data/egov-sample.json') as f:
        return json.load(f)

def save_as_sample_json(laws):
    with open('data/egov-sample.json', "w") as f:
        json.dump(laws, f, indent=2)

print("法令情報を取得中...")
laws = fetch_egov_laws(limit=100)



法令情報を取得中...
✓ 100件の法令情報を取得しました


In [4]:
laws

[{'law_number': '昭和二十一年憲法', 'law_name': '日本国憲法', 'text': '日本国憲法 (昭和二十一年憲法)'},
 {'law_number': '明治六年太政官布告第六十五号',
  'law_name': '明治六年太政官布告第六十五号（絞罪器械図式）',
  'text': '明治六年太政官布告第六十五号（絞罪器械図式） (明治六年太政官布告第六十五号)'},
 {'law_number': '明治十七年太政官布告第三十二号',
  'law_name': '明治十七年太政官布告第三十二号（爆発物取締罰則）',
  'text': '明治十七年太政官布告第三十二号（爆発物取締罰則） (明治十七年太政官布告第三十二号)'},
 {'law_number': '明治二十二年法律第三十四号',
  'law_name': '明治二十二年法律第三十四号（決闘罪ニ関スル件）',
  'text': '明治二十二年法律第三十四号（決闘罪ニ関スル件） (明治二十二年法律第三十四号)'},
 {'law_number': '明治二十三年法律第一号',
  'law_name': '保管金規則',
  'text': '保管金規則 (明治二十三年法律第一号)'},
 {'law_number': '明治二十八年法律第二十八号',
  'law_name': '通貨及証券模造取締法',
  'text': '通貨及証券模造取締法 (明治二十八年法律第二十八号)'},
 {'law_number': '明治二十九年法律第五号',
  'law_name': '国債証券買入銷却法',
  'text': '国債証券買入銷却法 (明治二十九年法律第五号)'},
 {'law_number': '明治二十九年法律第八十九号',
  'law_name': '民法',
  'text': '民法 (明治二十九年法律第八十九号)'},
 {'law_number': '明治三十年法律第二十九号',
  'law_name': '砂防法',
  'text': '砂防法 (明治三十年法律第二十九号)'},
 {'law_number': '明治三十一年法律第十一号',
  'law_name': '民法施行法',
  'text': '民法施行法 (明